In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
from io import BytesIO
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from numpy import mat, cov, mean, hstack, multiply,sqrt,diag, \
    squeeze, ones, array, vstack, kron, zeros, eye, savez_compressed
from numpy.linalg import inv
from scipy.stats import chi2
import yfinance as yf
from dateutil.relativedelta import relativedelta
import time
from ib_insync import *
from ib_insync import IB, util

In [2]:
# URL for FF5 factors from Fama-French website
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_CSV.zip'

# Fetch data from the URL
response = requests.get(url)

# Extract the content of the zip file
zip_content = response.content

# Open the zip archive
with zipfile.ZipFile(BytesIO(zip_content)) as myzip:
    # List all files in the archive for inspection
    file_list = myzip.namelist()
    
    # Find the correct file name (adjust based on what's listed)
    for file in file_list:
        if '5_Factors' in file:  # Adjust the condition to match the actual file name or pattern
            correct_file_name = file
            break
    
    # Read the correct file into a pandas DataFrame
    with myzip.open(correct_file_name) as myfile:
        ff5_data = pd.read_csv(myfile, skiprows=3)

# Function to clean and convert date
def clean_and_convert_date(x):
    try:
        return pd.to_datetime(str(x)[:6], format='%Y%m')  # Convert x to string and then slice first 6 characters
    except ValueError:
        return pd.NaT  # Return NaT for invalid dates

# Clean and convert the 'Unnamed: 0' column
ff5_data['Unnamed: 0'] = ff5_data['Unnamed: 0'].apply(clean_and_convert_date)

# Filter out rows with NaT (invalid dates)
ff5_data = ff5_data.dropna()

# Filter data for the last 20 years (from 2024 back to 2004)
current_year = pd.Timestamp.now().year
start_year = current_year - 20
end_year = pd.Timestamp.now().year

fixed_ff5_data = ff5_data[(ff5_data['Unnamed: 0'].dt.year >= start_year) & (ff5_data['Unnamed: 0'].dt.year <= end_year)]

# Reset index starting from 0 and rename 'Unnamed: 0' to 'Date'
fixed_ff5_data.reset_index(drop=True, inplace=True)
fixed_ff5_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Display the first few rows of the fixed dataframe with updated index and column name
fixed_ff5_data

C:\Users\priym\AppData\Local\Temp\ipykernel_11524\2534494187.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixed_ff5_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)


,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2004-01-01,2.15,2.45,2.49,-3.66,3.38,0.07
1,2004-02-01,1.40,-0.93,0.90,2.17,-1.21,0.06
2,2004-03-01,-1.32,2.10,0.27,1.56,-0.98,0.09
3,2004-04-01,-1.83,-2.01,-3.10,3.45,-2.84,0.08
4,2004-05-01,1.17,-0.38,-0.25,-1.18,0.03,0.06
...,...,...,...,...,...,...,...
239,2023-12-01,4.87,7.32,4.93,-3.07,1.32,0.43
240,2024-01-01,0.70,-5.74,-2.38,0.69,-0.96,0.47
241,2024-02-01,5.06,-0.78,-3.49,-1.99,-2.14,0.42
242,2024-03-01,2.83,-1.16,4.19,1.50,1.17,0.43


In [3]:
#IMPORTING Tickers S&P500

df_tickers = pd.read_excel('Scraping_Program\Scraping_Updated\ExcelFiles\Results.xlsx')
#df_tickers.head()

list_tickers = df_tickers['Ticker'].tolist()
print(list_tickers)

['MMM', 'ABT', 'ABBV', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALB', 'ARE', 'ALGN', 'LNT', 'ALL', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'ADI', 'ANSS', 'APA', 'AAPL', 'AMAT', 'ADM', 'ANET', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'WRB', 'BBY', 'TECH', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BLDR', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'CVX', 'CMG', 'CB', 'CHD', 'CINF', 'CTAS', 'CSCO', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CL', 'CMA', 'CAG', 'COP', 'ED', 'CEG', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DECK', 'DE', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DD', 'EMN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV', 'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EF

In [4]:
# List of tickers
tickers = list_tickers

# Get today's date
end_date = datetime.today()

# Calculate the date 20 years ago from today
try:
    start_date = end_date.replace(year=end_date.year - 20)
except ValueError:
    # Handle the case for leap years (if today is Feb 29 and 20 years ago is not a leap year)
    start_date = end_date.replace(year=end_date.year - 20, day=28)

# Format dates as strings in the required format
end_date_str = end_date.strftime('%Y-%m-%d')
start_date_str = start_date.strftime('%Y-%m-%d')

# Download stock prices from 20 years ago until today
data = yf.download(tickers, start=start_date_str, end=end_date_str, interval='1mo')

# Take only Adj Close
close_prices = data['Adj Close']

# Calculate monthly returns for each ticker
monthly_returns = close_prices.pct_change()

# Print first 5 monthly returns for each ticker
monthly_returns.reset_index(inplace=True)
monthly_returns.rename(columns={'index': 'Date'}, inplace=True)
monthly_returns

[*********************100%%**********************]  409 of 409 completed
C:\Users\priym\AppData\Local\Temp\ipykernel_11524\3217021041.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_returns.reset_index(inplace=True)


Ticker,Date,A,AAL,AAPL,ABBV,ABT,ADBE,ADI,ADM,ADP,...,WMT,WRB,WRK,WST,WYNN,XEL,XOM,XYL,YUM,ZBH
0,2004-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-07-01,-0.186817,NaN,-0.006146,NaN,-0.034593,-0.092634,-0.156754,-0.080454,0.005526,...,0.009714,-0.044444,NaN,-0.099527,-0.073777,0.036084,0.042558,NaN,0.031435,-0.134807
2,2004-08-01,-0.139017,NaN,0.066481,NaN,0.066329,0.087482,-0.125441,0.034997,-0.052644,...,-0.006414,-0.013678,NaN,0.055394,0.078815,0.032163,-0.004319,NaN,0.037187,-0.065653
3,2004-09-01,0.052195,NaN,0.123513,NaN,0.016071,0.078483,0.118821,0.068260,0.038974,...,0.017280,0.044081,NaN,0.042761,0.339119,-0.018697,0.054660,NaN,0.023923,0.108556
4,2004-10-01,0.161799,NaN,0.352258,NaN,0.006374,0.132905,0.038163,0.140754,0.053686,...,0.013534,0.016294,NaN,0.098801,0.124977,-0.000798,0.018415,NaN,0.069848,-0.018345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2024-02-01,0.055803,0.101897,-0.019794,0.070864,0.048520,-0.093075,-0.002807,-0.044440,0.021767,...,0.064024,0.021006,0.124938,-0.039327,0.114053,-0.119927,0.016633,0.129936,0.068963,-0.009873
237,2024-03-01,0.059333,-0.021046,-0.051286,0.034365,-0.041976,-0.099379,0.031123,0.182640,-0.005535,...,0.026617,0.057895,0.091853,0.104225,-0.028232,0.020118,0.112132,0.017237,0.001662,0.061274
238,2024-04-01,-0.058209,-0.119870,-0.006706,-0.106864,-0.067658,-0.082778,0.014258,-0.066072,-0.031433,...,-0.013628,-0.129692,-0.030131,-0.096611,-0.103492,-0.000372,0.017464,0.011297,0.018752,-0.088650
239,2024-05-01,-0.048380,-0.148779,0.128691,-0.008608,-0.035670,-0.039042,0.168885,0.064439,0.012526,...,0.108003,0.052748,0.118432,-0.072927,0.035243,0.032012,-0.008540,0.078960,-0.027044,-0.042650


In [5]:
# Assuming monthly_returns and fixed_ff5_data are your DataFrames
# Convert 'Date' columns to datetime format
monthly_returns['Date'] = pd.to_datetime(monthly_returns['Date'])
fixed_ff5_data['Date'] = pd.to_datetime(fixed_ff5_data['Date'])

# Identify the common dates (values) in 'Date' column
common_dates = set(monthly_returns['Date']).intersection(set(fixed_ff5_data['Date']))

# Filter both DataFrames based on common dates
monthly_returns = monthly_returns[monthly_returns['Date'].isin(common_dates)]
fixed_ff5_data = fixed_ff5_data[fixed_ff5_data['Date'].isin(common_dates)]

# Set 'Date' columns as index (if needed)
monthly_returns.set_index('Date', inplace=True)
fixed_ff5_data.set_index('Date', inplace=True)

# Concatenate the DataFrames
df_conc = pd.concat([monthly_returns, fixed_ff5_data], axis=1)

# Replace NaN values with 0
df_conc = df_conc.fillna(0)

# Display the resulting DataFrame
df_conc


C:\Users\priym\AppData\Local\Temp\ipykernel_11524\553945071.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixed_ff5_data['Date'] = pd.to_datetime(fixed_ff5_data['Date'])


,A,AAL,AAPL,ABBV,ABT,ADBE,ADI,ADM,ADP,ADSK,...,XOM,XYL,YUM,ZBH,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.86,2.56,1.18,1.20,-0.39,0.08
2004-07-01,-0.186817,0.000000,-0.006146,0.000000,-0.034593,-0.092634,-0.156754,-0.080454,0.005526,-0.060298,...,0.042558,0.000000,0.031435,-0.134807,-4.06,-2.95,3.24,5.32,-1.66,0.10
2004-08-01,-0.139017,0.000000,0.066481,0.000000,0.066329,0.087482,-0.125441,0.034997,-0.052644,0.104726,...,-0.004319,0.000000,0.037187,-0.065653,0.08,-1.13,0.97,1.23,-1.47,0.11
2004-09-01,0.052195,0.000000,0.123513,0.000000,0.016071,0.078483,0.118821,0.068260,0.038974,0.095024,...,0.054660,0.000000,0.023923,0.108556,1.60,3.27,0.00,-1.49,-1.88,0.11
2004-10-01,0.161799,0.000000,0.352258,0.000000,0.006374,0.132905,0.038163,0.140754,0.053686,0.085394,...,0.018415,0.000000,0.069848,-0.018345,1.43,0.19,-0.22,-0.52,0.49,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,0.087872,0.105390,0.013583,0.088349,0.055422,-0.023584,0.082779,-0.020480,0.013265,0.114682,...,-0.026864,0.087796,0.040701,0.046342,4.87,7.32,4.93,-3.07,1.32,0.43
2024-01-01,-0.064231,0.035662,-0.042227,0.060850,0.027982,0.035501,-0.031225,-0.230407,0.054986,0.042426,...,0.028306,-0.016789,-0.008955,0.032046,0.70,-5.74,-2.38,0.69,-0.96,0.47
2024-02-01,0.055803,0.101897,-0.019794,0.070864,0.048520,-0.093075,-0.002807,-0.044440,0.021767,0.017178,...,0.016633,0.129936,0.068963,-0.009873,5.06,-0.78,-3.49,-1.99,-2.14,0.42


In [6]:
# Initialize df_excess_returns with the same structure as monthly_returns
df_excess_returns = monthly_returns.copy()

# Get the 'RF' column (last column)
rf_column = df_conc.iloc[:, -1]

# Convert rf_column to numeric
rf_column = pd.to_numeric(rf_column, errors='coerce')


# Calculate the excess returns
for column in df_excess_returns.columns:  # Excluding Date and the last 6 columns (Factors)
    mask = ~df_excess_returns[column].isna()
    df_excess_returns.loc[mask, column] = df_excess_returns.loc[mask, column] - rf_column[mask]

df_excess_returns = df_excess_returns.fillna(0)
df_excess_returns

Ticker,A,AAL,AAPL,ABBV,ABT,ADBE,ADI,ADM,ADP,ADSK,...,WMT,WRB,WRK,WST,WYNN,XEL,XOM,XYL,YUM,ZBH
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2004-07-01,-0.286817,0.000000,-0.106146,0.000000,-0.134593,-0.192634,-0.256754,-0.180454,-0.094474,-0.160298,...,-0.090286,-0.144444,0.000000,-0.199527,-0.173777,-0.063916,-0.057442,0.000000,-0.068565,-0.234807
2004-08-01,-0.249017,0.000000,-0.043519,0.000000,-0.043671,-0.022518,-0.235441,-0.075003,-0.162644,-0.005274,...,-0.116414,-0.123678,0.000000,-0.054606,-0.031185,-0.077837,-0.114319,0.000000,-0.072813,-0.175653
2004-09-01,-0.057805,0.000000,0.013513,0.000000,-0.093929,-0.031517,0.008821,-0.041740,-0.071026,-0.014976,...,-0.092720,-0.065919,0.000000,-0.067239,0.229119,-0.128697,-0.055340,0.000000,-0.086077,-0.001444
2004-10-01,0.051799,0.000000,0.242258,0.000000,-0.103626,0.022905,-0.071837,0.030754,-0.056314,-0.024606,...,-0.096466,-0.093706,0.000000,-0.011199,0.014977,-0.110798,-0.091585,0.000000,-0.040152,-0.128345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,-0.342128,-0.324610,-0.416417,-0.341651,-0.374578,-0.453584,-0.347221,-0.450480,-0.416735,-0.315318,...,-0.417411,-0.455224,-0.421499,-0.426123,-0.350753,-0.412413,-0.456864,-0.342204,-0.389299,-0.383658
2024-01-01,-0.534231,-0.434338,-0.512227,-0.409150,-0.442018,-0.434499,-0.501225,-0.700407,-0.415014,-0.427574,...,-0.421792,-0.312195,-0.500347,-0.410617,-0.433561,-0.502951,-0.441694,-0.486789,-0.478955,-0.437954
2024-02-01,-0.364197,-0.318103,-0.439794,-0.349136,-0.371480,-0.513075,-0.422807,-0.464440,-0.398233,-0.402822,...,-0.355976,-0.398994,-0.295062,-0.459327,-0.305947,-0.539927,-0.403367,-0.290064,-0.351037,-0.429873


In [7]:
# List of the tickers
tickers = list_tickers

# Dictionary in which to save the regression results:
regression_results_ff = {}

# Convert every column in the DataFrame to float
df_conc = df_conc.astype(float)

# Iterate over all tickers
for tickers_of_interest in tickers:
    # Dependent Variable: tickers excess return
    y_ff = df_excess_returns[tickers_of_interest]

    # Independent Variable: market excess return, SMB, HML with first column of ones
    X_ff = sm.add_constant(df_conc[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']])

    # Solve the equation using Matrix Algebra
    beta_ff = np.linalg.inv(X_ff.T @ X_ff) @ X_ff.T @ y_ff

    # Extract Alfas, Betas for Mkt-RF, Betas for SMB, Betas for HML, Betas for RMW, Betas for CMA
    alfa_ff = beta_ff[0]
    beta_mkt_rf_ff = beta_ff[1]
    beta_smb_ff = beta_ff[2]
    beta_hml_ff = beta_ff[3]
    beta_rmw_ff = beta_ff[4]
    beta_cma_ff = beta_ff[5]
    

    # Save the results in the Dictionary
    regression_results_ff[tickers_of_interest] = {
        'Alfa': alfa_ff,
        'Beta_Mkt-RF': beta_mkt_rf_ff,
        'Beta_SMB': beta_smb_ff,
        'Beta_HML': beta_hml_ff,
        'Beta_RMW': beta_rmw_ff,
        'Beta_CMA': beta_cma_ff
    }

# Print the results (alphas and betas) for each ticker
for tickers_of_interest, results in regression_results_ff.items():
    print(f'Results for portfolio {tickers_of_interest}:')
    print(f'Alfa (α): {results["Alfa"]}')
    print(f'Beta_Mkt-RF (β): {results["Beta_Mkt-RF"]}')
    print(f'Beta_SMB (β): {results["Beta_SMB"]}')
    print(f'Beta_HML (β): {results["Beta_HML"]}')
    print(f'Beta_RMW (β): {results["Beta_RMW"]}')
    print(f'Beta_CMA (β): {results["Beta_CMA"]}')
    print()

Results for portfolio MMM:
Alfa (α): -0.12627691320157403
Beta_Mkt-RF (β): 0.008254350291321632
Beta_SMB (β): 0.007879363102502528
Beta_HML (β): -0.009440373572663913
Beta_RMW (β): 0.01037336439601096
Beta_CMA (β): 0.31801142740239857

Results for portfolio ABT:
Alfa (α): -0.11851484376883095
Beta_Mkt-RF (β): 0.004983907660109707
Beta_SMB (β): 0.002135140498768438
Beta_HML (β): -0.015385322011072703
Beta_RMW (β): 0.006476272634613035
Beta_CMA (β): 0.45184639987828334

Results for portfolio ABBV:
Alfa (α): -0.053016545252264714
Beta_Mkt-RF (β): 0.002152552352432589
Beta_SMB (β): 0.007909512051332132
Beta_HML (β): -0.00520435483552043
Beta_RMW (β): 0.0077238116887635815
Beta_CMA (β): 0.18194115662441604

Results for portfolio ADBE:
Alfa (α): -0.11513911011974136
Beta_Mkt-RF (β): 0.01453978628445636
Beta_SMB (β): 0.0035907138843792206
Beta_HML (β): -0.007198951328596193
Beta_RMW (β): 0.001829564241231608
Beta_CMA (β): 0.11532591696961968

Results for portfolio AMD:
Alfa (α): -0.1133256098

In [8]:
# Remove duplicate labels from df_conc
df_conc = df_conc.loc[:, ~df_conc.columns.duplicated()]

# Dictionary with predicted Fama-French Data for each Portfolio
predicted_data_ff = {
    'Mkt-RF': df_conc['Mkt-RF'],
    'SMB': df_conc['SMB'],
    'HML': df_conc['HML'],
    'RMW': df_conc['RMW'],
    'CMA': df_conc['CMA']
}

# Add predicted Fama-French Data for each Portfolio to the Dictionary 
for tickers_of_interest in tickers:
    predicted_data_ff[tickers_of_interest] = (
        regression_results_ff[tickers_of_interest]['Alfa'] +
        regression_results_ff[tickers_of_interest]['Beta_Mkt-RF'] * df_conc['Mkt-RF'] +
        regression_results_ff[tickers_of_interest]['Beta_SMB'] * df_conc['SMB'] +
        regression_results_ff[tickers_of_interest]['Beta_HML'] * df_conc['HML'] +
        regression_results_ff[tickers_of_interest]['Beta_RMW'] * df_conc['RMW'] +
        regression_results_ff[tickers_of_interest]['Beta_CMA'] * df_conc['CMA']
    )

# Create a Data Frame using the Dictionary 
df_predicted_data_ff = pd.DataFrame(predicted_data_ff)

# Keep only the predicted Fama-French Data for each stock
df_predicted_data_ff = pd.DataFrame(predicted_data_ff)
df_predicted_data_ff = df_predicted_data_ff.iloc[:, 5:]
print(df_predicted_data_ff)


                 MMM       ABT      ABBV      ADBE       AMD       AES  \
Date                                                                     
2004-06-01 -0.089444 -0.114162 -0.025637 -0.085202 -0.065592 -0.088276   
2004-07-01 -0.134483 -0.126411 -0.047157 -0.189669 -0.268064 -0.187707   
2004-08-01 -0.121781 -0.114504 -0.052102 -0.119452 -0.132063 -0.129584   
2004-09-01 -0.106990 -0.119218 -0.037637 -0.084394 -0.048408 -0.087291   
2004-10-01 -0.101857 -0.090454 -0.043048 -0.087797 -0.070239 -0.096216   
...              ...       ...       ...       ...       ...       ...   
2023-12-01 -0.032314 -0.068529  0.008603 -0.032146  0.030728 -0.016906   
2024-01-01 -0.154506 -0.112347 -0.089724 -0.113851 -0.127237 -0.148317   
2024-02-01 -0.097706 -0.081654 -0.056574 -0.029904  0.031637 -0.059997   
2024-03-01 -0.099923 -0.110304 -0.045650 -0.092474 -0.103133 -0.108129   
2024-04-01 -0.192728 -0.169473 -0.085178 -0.195944 -0.246685 -0.204701   

                 AFL         A       

## PRICE TODAY

In [9]:
# Assuming df_predicted_data_ff is your DataFrame
date_list = df_predicted_data_ff.index.tolist()
print(date_list)


[Timestamp('2004-06-01 00:00:00'), Timestamp('2004-07-01 00:00:00'), Timestamp('2004-08-01 00:00:00'), Timestamp('2004-09-01 00:00:00'), Timestamp('2004-10-01 00:00:00'), Timestamp('2004-11-01 00:00:00'), Timestamp('2004-12-01 00:00:00'), Timestamp('2005-01-01 00:00:00'), Timestamp('2005-02-01 00:00:00'), Timestamp('2005-03-01 00:00:00'), Timestamp('2005-04-01 00:00:00'), Timestamp('2005-05-01 00:00:00'), Timestamp('2005-06-01 00:00:00'), Timestamp('2005-07-01 00:00:00'), Timestamp('2005-08-01 00:00:00'), Timestamp('2005-09-01 00:00:00'), Timestamp('2005-10-01 00:00:00'), Timestamp('2005-11-01 00:00:00'), Timestamp('2005-12-01 00:00:00'), Timestamp('2006-01-01 00:00:00'), Timestamp('2006-02-01 00:00:00'), Timestamp('2006-03-01 00:00:00'), Timestamp('2006-04-01 00:00:00'), Timestamp('2006-05-01 00:00:00'), Timestamp('2006-06-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2006-08-01 00:00:00'), Timestamp('2006-09-01 00:00:00'), Timestamp('2006-10-01 00:00:00'), Timestamp('20

In [10]:
# List of the tickers
tickers = list_tickers

end_date = date_list[-1]  # Second-to-last date
start_date = date_list[-2]  # Date before the second-to-last date

# Download data for each stock from Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')

# Take only today's Adj Close
close_prices = data['Adj Close']

print(close_prices)

[*********************100%%**********************]  409 of 409 completed


Ticker               A    AAL        AAPL        ABBV         ABT        ADBE  \
Date                                                                            
2024-03-01  145.509995  15.35  171.479996  182.100006  113.660004  504.600006   

Ticker             ADI        ADM         ADP        ADSK  ...        WMT  \
Date                                                       ...              
2024-03-01  197.789993  62.810001  249.740005  260.420013  ...  60.169998   

Ticker            WRB        WRK         WST        WYNN    XEL         XOM  \
Date                                                                          
2024-03-01  88.440002  49.450001  395.709991  102.230003  53.75  116.239998   

Ticker             XYL         YUM         ZBH  
Date                                            
2024-03-01  129.240005  138.649994  131.979996  

[1 rows x 409 columns]


In [11]:
# Replace NaN values with 0
close_pricess = close_prices.fillna(0)
# Keep the last predicted return for each stock
last_return = df_predicted_data_ff.iloc[-1]
# Make transpose 
close_pricesss = close_pricess.transpose()
# Data Frame with close prices from month ago compared to the last predicted return for each stock and the last return
merged_df = pd.concat([close_pricesss, last_return], axis=1)
# Rename columns
new_names_columns = ['Month_Ago_Price','Last_ff5_returns']
merged_df.columns = new_names_columns
print(merged_df)

      Month_Ago_Price  Last_ff5_returns
A          145.509995         -0.212922
AAL         15.350000         -0.201065
AAPL       171.479996         -0.173587
ABBV       182.100006         -0.085178
ABT        113.660004         -0.169473
...               ...               ...
XEL         53.750000         -0.148186
XOM        116.239998         -0.173869
XYL        129.240005         -0.092198
YUM        138.649994         -0.176035
ZBH        131.979996         -0.202004

[409 rows x 2 columns]


# **FINDING PRICE FOR A MONTH FROM THE LATEST ENTRY IN FAMA FRENCH DATE USING PREDICTED BY FAMA FRENCH 5 FACTORS RETURNS**
#### EXAMPLE OF THE LOGIC
#### RET(2024/04) = P(2024/04) / P(2024/03) - 1
#### P(2024/04) = P(2024/03) * (1 + RET(2024/04))

In [12]:
# Calculate the predicted price for each stock on 28/03/2024
merged_df['Next_Month'] = merged_df['Month_Ago_Price'] * (1 + merged_df['Last_ff5_returns'])

# Keep only the next month column, containing the predicted price for each stock
limit_prices_2024 = merged_df['Next_Month']
print(limit_prices_2024)

A       114.527647
AAL      12.263645
AAPL    141.713284
ABBV    166.589170
ABT      94.397694
           ...    
XEL      45.784986
XOM      96.029456
XYL     117.324387
YUM     114.242740
ZBH     105.319557
Name: Next_Month, Length: 409, dtype: float64


## COMPARISON BETWEEN REAL AND PREDICTED PRICE 2024/03/28 ##

In [13]:
# List of the tickers
tickers = list_tickers

latest_date = max(date_list)  # Find the latest date in date_list
end_date = latest_date + relativedelta(months=1) # Add one month to the latest date
start_date = date_list[-1]  # Date before the second-to-last date

# Download data for each stock from Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')

# Take only Adj Close 28/03/2024
close_prices1 = data['Adj Close']

print(close_prices1)

[*********************100%%**********************]  409 of 409 completed

Ticker               A    AAL        AAPL        ABBV         ABT        ADBE  \
Date                                                                            
2024-04-01  137.039993  13.51  170.330002  162.639999  105.970001  462.829987   

Ticker             ADI    ADM         ADP        ADSK  ...        WMT  \
Date                                                   ...              
2024-04-01  200.610001  58.66  241.889999  212.850006  ...  59.349998   

Ticker            WRB        WRK         WST       WYNN    XEL         XOM  \
Date                                                                         
2024-04-01  76.970001  47.959999  357.480011  91.650002  53.73  118.269997   

Ticker             XYL     YUM         ZBH  
Date                                        
2024-04-01  130.699997  141.25  120.279999  

[1 rows x 409 columns]


In [14]:
# Replace NaN values with 0
close_pricess1 = close_prices1.fillna(0)
# Make transpose
close_pricesss1 = close_pricess1.transpose()
# Merging the Data Frame with closed prices and predicted price for each stock
merged_df1 = pd.concat([close_pricesss1, merged_df['Next_Month']], axis=1)
merged_df1

,2024-04-01 00:00:00,Next_Month
A,137.039993,114.527647
AAL,13.510000,12.263645
AAPL,170.330002,141.713284
ABBV,162.639999,166.589170
ABT,105.970001,94.397694
...,...,...
XEL,53.730000,45.784986
XOM,118.269997,96.029456
XYL,130.699997,117.324387
YUM,141.250000,114.242740


In [15]:
# Rename the columns (real price and fama french predicted price 28/03/2024) 
merged_df1.columns = ['real_price', 'predicted_price']

# Determine "buy" or "sell" on the basis of real price and predicted price for today
def determine_action(row):
    if row['predicted_price'] < row['real_price']:
        return "sell"
    else:
        return "buy"

# Apply the fuction to each row of the DataFrame (each stock)
merged_df1['action'] = merged_df1.apply(determine_action, axis=1)

# Print the DataFrame with the action for each stock
print(merged_df1[['real_price', 'predicted_price', 'action']])



      real_price  predicted_price action
A     137.039993       114.527647   sell
AAL    13.510000        12.263645   sell
AAPL  170.330002       141.713284   sell
ABBV  162.639999       166.589170    buy
ABT   105.970001        94.397694   sell
...          ...              ...    ...
XEL    53.730000        45.784986   sell
XOM   118.269997        96.029456   sell
XYL   130.699997       117.324387   sell
YUM   141.250000       114.242740   sell
ZBH   120.279999       105.319557   sell

[409 rows x 3 columns]


## TICKER - ACTION - LIMIT PRICE

In [16]:
# DataFrame with the action for each stock and the predicted price for each stock
final_dataframe = pd.concat([merged_df1['action'] , limit_prices_2024], axis=1)
final_dataframe

,action,Next_Month
A,sell,114.527647
AAL,sell,12.263645
AAPL,sell,141.713284
ABBV,buy,166.589170
ABT,sell,94.397694
...,...,...
XEL,sell,45.784986
XOM,sell,96.029456
XYL,sell,117.324387
YUM,sell,114.242740


## Portfolio Formation ##

In [17]:
#IMPORTING Scraping Results
df_results = pd.read_excel('Scraping_Program\Scraping_Updated\ExcelFiles\Results.xlsx')

# Putting column "Ticker" as row index 
df_results.set_index('Ticker', inplace=True)

# Print DataFrame with "Ticker" column as index 
print("\nDataFrame with 'Ticker' as row index:")
print(df_results)



DataFrame with 'Ticker' as row index:
       Decision  WeightDiff
Ticker                     
MMM        SELL   -0.001402
ABT        SELL   -0.042773
ABBV       SELL   -0.258190
ADBE       SELL   -0.105221
AMD        SELL   -0.147948
...         ...         ...
WYNN       SELL   -0.000135
XEL        SELL   -0.030936
XYL        SELL   -0.001280
YUM        SELL   -0.032483
ZBH        SELL   -0.015566

[409 rows x 2 columns]


In [18]:
# Union of DataFrames keeping only common indexes
merged_df = df_results.join(final_dataframe, how='inner')

# Print merged_df
#print(merged_df)

# Filtering rows in which "Decision" and "action" match
filtered_merged_df = merged_df[((merged_df['Decision'] == 'SELL') & (merged_df['action'] == 'sell')) |
                               ((merged_df['Decision'] == 'BUY') & (merged_df['action'] == 'buy'))]

# Print filtered DataFrame
print("\nFiltered DataFrame:")
print(filtered_merged_df)



Filtered DataFrame:
     Decision  WeightDiff action  Next_Month
MMM      SELL   -0.001402   sell   71.594758
ABT      SELL   -0.042773   sell   94.397694
ADBE     SELL   -0.105221   sell  405.726715
AMD      SELL   -0.147948   sell  135.965820
AES      SELL   -0.001756   sell   14.259707
...       ...         ...    ...         ...
WYNN     SELL   -0.000135   sell   77.076911
XEL      SELL   -0.030936   sell   45.784986
XYL      SELL   -0.001280   sell  117.324387
YUM      SELL   -0.032483   sell  114.242740
ZBH      SELL   -0.015566   sell  105.319557

[290 rows x 4 columns]


In [19]:
# Filter for buy positions only (WeightDiff > 0)
buy_positions = filtered_merged_df[filtered_merged_df['WeightDiff'] > 0]

# Filter for sell positions only (WeightDiff < 0)
sell_positions = filtered_merged_df[filtered_merged_df['WeightDiff'] < 0]

# First 5 rows with highest values in 'WeightDiff' for buy positions
top_5_high = buy_positions.nlargest(5, 'WeightDiff')

# Last 5 rows with lowest values in 'WeightDiff' for sell positions
bottom_5_low = sell_positions.nsmallest(5, 'WeightDiff')
print("First 5 rows with highest values in 'WeightDiff':")
print(top_5_high)
print("\nLast 5 rows with the lowest values in 'WeightDiff':")
print(bottom_5_low)


First 5 rows with highest values in 'WeightDiff':
    Decision  WeightDiff action  Next_Month
FTV      BUY    0.004481    buy   78.021222
CZR      BUY    0.001074    buy   37.892311

Last 5 rows with the lowest values in 'WeightDiff':
    Decision  WeightDiff action  Next_Month
IBM     SELL   -2.527826   sell  156.594939
PNC     SELL   -0.505860   sell  129.220540
JPM     SELL   -0.404503   sell  160.755109
XOM     SELL   -0.296463   sell   96.029456
ADP     SELL   -0.284645   sell  206.442250


In [20]:
# Calculate the sum of the values in 'WeightDiff' for the first 5 rows with the highest values 
sum_all = top_5_high['WeightDiff'].sum() + bottom_5_low['WeightDiff'].sum()

# Normalize the weights for the first 5 rows with the highest values
top_5_high['WeightDiff_normalized'] = top_5_high['WeightDiff'] / sum_all

# Normalize the weights for the last 5 rows with the lowest values 
bottom_5_low['WeightDiff_normalized'] = bottom_5_low['WeightDiff'] / sum_all

# Print the normalized weights for the first 5 rows with the highest values
print("Normalized weights for the first 5 rows with highest values:")
print(top_5_high[['WeightDiff', 'WeightDiff_normalized', 'Decision', 'Next_Month']])
# Print the normalized weights for the last 5 rows with the lowest values
print("\nNormalized weights for the last 5 rows with the lowest values:")
print(bottom_5_low[['WeightDiff', 'WeightDiff_normalized', 'Decision', 'Next_Month']])


Normalized weights for the first 5 rows with highest values:
     WeightDiff  WeightDiff_normalized Decision  Next_Month
FTV    0.004481              -0.001116      BUY   78.021222
CZR    0.001074              -0.000268      BUY   37.892311

Normalized weights for the last 5 rows with the lowest values:
     WeightDiff  WeightDiff_normalized Decision  Next_Month
IBM   -2.527826               0.629793     SELL  156.594939
PNC   -0.505860               0.126032     SELL  129.220540
JPM   -0.404503               0.100780     SELL  160.755109
XOM   -0.296463               0.073862     SELL   96.029456
ADP   -0.284645               0.070918     SELL  206.442250


# Part II: Algotrading

## Final dataframe
This is the final dataframe which will go into our algotrading code

In [21]:
longtable = top_5_high[['WeightDiff', 'WeightDiff_normalized', 'Decision', 'Next_Month']]
print(longtable)
shorttable = bottom_5_low[['WeightDiff', 'WeightDiff_normalized', 'Decision', 'Next_Month']]
print(shorttable)

     WeightDiff  WeightDiff_normalized Decision  Next_Month
FTV    0.004481              -0.001116      BUY   78.021222
CZR    0.001074              -0.000268      BUY   37.892311
     WeightDiff  WeightDiff_normalized Decision  Next_Month
IBM   -2.527826               0.629793     SELL  156.594939
PNC   -0.505860               0.126032     SELL  129.220540
JPM   -0.404503               0.100780     SELL  160.755109
XOM   -0.296463               0.073862     SELL   96.029456
ADP   -0.284645               0.070918     SELL  206.442250


In [22]:
# Concatenate the DataFrames with the portfolio weights to use it for trader
merged_table = pd.concat([longtable, shorttable])
merged_table.to_csv('final_dataframe.csv')
merged_table
# Calculate the sum of the normalized weights to chek if it is equal to 1
#sm= sum(merged_table['WeightDiff_normalized'])
#sm

,WeightDiff,WeightDiff_normalized,Decision,Next_Month
FTV,0.004481,-0.001116,BUY,78.021222
CZR,0.001074,-0.000268,BUY,37.892311
IBM,-2.527826,0.629793,SELL,156.594939
PNC,-0.505860,0.126032,SELL,129.220540
JPM,-0.404503,0.100780,SELL,160.755109
XOM,-0.296463,0.073862,SELL,96.029456
ADP,-0.284645,0.070918,SELL,206.442250


In [23]:
data = pd.read_csv("final_dataframe.csv")
data = data.rename(columns={data.columns[0]: "ticker",data.columns[4]: "fair_prices"})
# Round 'fair_prices' column to two decimal places
data['fair_prices'] = data['fair_prices'].round(2)
data

,ticker,WeightDiff,WeightDiff_normalized,Decision,fair_prices
0,FTV,0.004481,-0.001116,BUY,78.02
1,CZR,0.001074,-0.000268,BUY,37.89
2,IBM,-2.527826,0.629793,SELL,156.59
3,PNC,-0.505860,0.126032,SELL,129.22
4,JPM,-0.404503,0.100780,SELL,160.76
5,XOM,-0.296463,0.073862,SELL,96.03
6,ADP,-0.284645,0.070918,SELL,206.44


### Connect to the paper trading account

##### You can run this code only during trading hours (15:30 CET) and with Interative Brockers launched

In [24]:
# Starts a loop of the event processing
util.startLoop()
# Account Summary
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=0) #Port 7496 is reserved for real account, while 7497 for demo

<IB connected to 127.0.0.1:7497 clientId=0>

#### Save the account value below to enter it into the Cancellation code ####

In [25]:
myAccount = ib.accountSummary()
#myAccount
# Positions in the portfolio of the IB account
# ib.positions()
myportfoliovalue_begin_tcv = myAccount[29].value # total cash value of the account we use to trade
print(myportfoliovalue_begin_tcv) # save this value to compare with the end value of the portfolio

In [26]:
# Calculate the quantity to buy/sell for each stock based on the total cash value of the IB account
myportfoliovalue_begin_tcv = pd.to_numeric(myportfoliovalue_begin_tcv)

data['quantity'] = round (abs ((myportfoliovalue_begin_tcv / 2) * data['WeightDiff_normalized'] / data['fair_prices']))
data

,ticker,WeightDiff,WeightDiff_normalized,Decision,fair_prices,quantity
0,FTV,0.004481,-0.001116,BUY,78.02,7.0
1,CZR,0.001074,-0.000268,BUY,37.89,4.0
2,IBM,-2.527826,0.629793,SELL,156.59,2034.0
3,PNC,-0.505860,0.126032,SELL,129.22,493.0
4,JPM,-0.404503,0.100780,SELL,160.76,317.0
5,XOM,-0.296463,0.073862,SELL,96.03,389.0
6,ADP,-0.284645,0.070918,SELL,206.44,174.0


In [27]:
for i in range(len(data)):
    # Create a Stock contract for the current ticker
    contract = Stock(symbol=data['ticker'].iloc[i], exchange='SMART', currency='USD')
      
    # Qualify the contract
    ib.qualifyContracts(contract)
        
    # Create a Market Order object based on 'Decision' and 'quantity'
    marketOrder = MarketOrder(action=data['Decision'].iloc[i], totalQuantity=data['quantity'].iloc[i])
        
    # Place the market order
    trade = ib.placeOrder(contract, marketOrder)
        
    # Print confirmation message
    print(f"Placed order for {data['ticker'].iloc[i]}: {data['Decision'].iloc[i]} {data['quantity'].iloc[i]} shares.")
        

Placed order for FTV: BUY 7.0 shares.
Placed order for CZR: BUY 4.0 shares.
Placed order for IBM: SELL 2034.0 shares.
Placed order for PNC: SELL 493.0 shares.
Placed order for JPM: SELL 317.0 shares.
Placed order for XOM: SELL 389.0 shares.
Placed order for ADP: SELL 174.0 shares.


Error 383, reqId 605: The following order "ID:605" size exceeds the Size Limit of 500. Restriction is specified in Precautionary Settings of Global Configuration/Presets.
Canceled order: Trade(contract=Stock(conId=8314, symbol='IBM', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='IBM', tradingClass='IBM'), order=MarketOrder(orderId=605, action='SELL', totalQuantity=2034.0), orderStatus=OrderStatus(orderId=605, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 21, 16, 55, 37, 212275, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 21, 16, 55, 37, 220284, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 383, reqId 605: The following order "ID:605" size exceeds the Size Limit of 500. Restriction is specified in Pre

In [28]:
# Iterate through each row in the data DataFrame
for i in range(len(data)):
    # Create a Stock contract object for the stock
    contract = Stock(symbol=data.loc[i, 'ticker'], exchange='SMART', currency='USD')
    
    # Determine the opposite action for the limit order
    opposite_action = 'SELL' if data.loc[i, 'Decision'] == 'BUY' else 'BUY'
    
    # Create a LimitOrder object with the opposite action, quantity, and fair price
    limit_order = LimitOrder(action=opposite_action,
                             totalQuantity=data.loc[i, 'quantity'],
                             lmtPrice=data.loc[i, 'fair_prices'])
    
    # Qualify the contract (fill in the contract details)
    qualified_contract = ib.qualifyContracts(contract)[0]
    
    # Place the limit order
    trade = ib.placeOrder(qualified_contract, limit_order)
    
    # Print order details for verification
    print(f"Setting up limit order for {data.loc[i, 'ticker']}: {opposite_action} {data.loc[i, 'quantity']} shares at limit price ${data.loc[i, 'fair_prices']}")


Setting up limit order for FTV: SELL 7.0 shares at limit price $78.02
Setting up limit order for CZR: SELL 4.0 shares at limit price $37.89
Setting up limit order for IBM: BUY 2034.0 shares at limit price $156.59
Setting up limit order for PNC: BUY 493.0 shares at limit price $129.22
Setting up limit order for JPM: BUY 317.0 shares at limit price $160.76
Setting up limit order for XOM: BUY 389.0 shares at limit price $96.03
Setting up limit order for ADP: BUY 174.0 shares at limit price $206.44
